This lab on Ridge Regression and the Lasso is a Python adaptation of p. 251-255 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. Based on the work of R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016).

# 6.6: Ridge Regression and the Lasso

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, scale
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

We will use the `sklearn` package in order to perform ridge regression and
the lasso. The main functions in this package that we care about are `Ridge()`, which can be used
to fit ridge regression models, and `Lasso()` which will fit lasso models. They also have cross-validated counterparts: `RidgeCV()` and `LassoCV()`. We'll use these a bit later.

Before proceeding, let's first ensure that the missing values have
been removed from the data, as described in the previous lab.

In [ ]:
use_smith_edu = False  or True
if use_smith_edu:
    url = ''
    df = pd.read_csv('https://www.science.smith.edu/~jcrouser/SDS293/data/Hitters.csv')
    df = df.set_index('Player')
else:
    df = smd.get_rdataset('Hitters', 'ISLR')
    df.data.__doc__ = df.__doc__
    df = df.data

df = df.dropna()
df.head()


We will now perform ridge regression and the lasso in order to predict `Salary` on
the `Hitters` data. Let's set up our data:

In [ ]:
y = df.Salary

dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])

# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis = 1).astype('float64')

# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis = 1)

X.info()

# 6.6.1 Ridge Regression
The `Ridge()` function has an alpha argument ($\lambda$, but with a different name!) that is used to tune the model. We'll generate an array of alpha values ranging from very big to very small, essentially
covering the full range of scenarios from the null model containing
only the intercept, to the least squares fit:

In [ ]:
alphas = 10**np.linspace(10, -2, 100)
alphas

Associated with each alpha value is a vector of ridge regression coefficients, which we'll
store in a matrix `coefs`. In this case, it is a $19 \times 100$
matrix, with 19 rows (one for each predictor) and 100
columns (one for each value of alpha). Remember that we'll want to standardize the
variables so that they are on the same scale. To do this, we can use the
`normalize = True` parameter:

In [ ]:
ridge = Ridge()
pipeline = make_pipeline(StandardScaler(), ridge)
coefs = []
train_err = []

for a in alphas:
    ridge.set_params(alpha = a)
    pipeline.fit(X, y)
    train_err.append(mean_squared_error(y, pipeline.predict(X)))
    coefs.append(ridge.coef_)
    
np.shape(coefs)

We expect the coefficient estimates to be much smaller, in terms of $l_2$ norm,
when a large value of alpha is used, as compared to when a small value of alpha is
used. Let's plot and find out:

In [ ]:
# sns.scatterplot(alphas, coefs)
# ax = plt.gca()
plt.figure(figsize=(8,8))
plt.plot(alphas, coefs, label=X.columns)
plt.legend(loc='upper right')
plt.xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.grid()
plt.show()

In [ ]:
plt.plot(alphas, train_err)
plt.xscale('log')
plt.grid()
plt.show()

In [ ]:
plt.plot(alphas, np.linalg.norm(coefs,axis=1))
plt.xscale('log')
plt.yscale('log')
plt.grid()

We now split the samples into a training set and a test set in order
to estimate the test error of ridge regression and the lasso:

In [ ]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

Next we fit a ridge regression model on the training set, and evaluate
its MSE on the test set, using $\lambda = 4$:

In [ ]:
def print_result(estimator, X_test=None, y_test=None):
    intercept, coef = estimator[-1].intercept_, estimator[-1].coef_
    df = pd.DataFrame(coef[None], columns=estimator[0].feature_names_in_)
    df['intercept'] = intercept
    print('-'*20)
    print(f'alpha={estimator[-1].alpha}')
    # print(f'scaler mean {estimator["standardscaler"].mean_}')
    print('-'*20)
    print(df)
    if not(X_test is None or y_test is None):
        pred = estimator.predict(X=X_test)
        print('-'*20)
        print(f'MSE {mean_squared_error(y_test, pred):.2f}')
        print('-'*20)

ridge2 = make_pipeline(StandardScaler(with_mean=True, with_std=True), Ridge(alpha=4))

# Fit a ridge regression on the training data
ridge2.fit(X_train, y_train)

print_result(ridge2, X_test, y_test)


The test MSE when alpha = 4 is 106216. Now let's see what happens if we use a huge value of alpha, say $10^{10}$:

In [ ]:
ridge3 = make_pipeline(StandardScaler(), Ridge(alpha = 10**10))
ridge3.fit(X_train, y_train)             # Fit a ridge regression on the training data
print_result(ridge3, X_test, y_test)

This big penalty shrinks the coefficients to a very large degree, essentially reducing to a model containing just the intercept. This over-shrinking makes the model more biased, resulting in a higher MSE.

Okay, so fitting a ridge regression model with alpha = 4 leads to a much lower test
MSE than fitting a model with just an intercept. We now check whether
there is any benefit to performing ridge regression with alpha = 4 instead of
just performing least squares regression. Recall that least squares is simply
ridge regression with alpha = 0.

In [ ]:
ridge2 = make_pipeline(StandardScaler(), Ridge(alpha = 0))
ridge2.fit(X_train, y_train)             # Fit a ridge regression on the training data

print_result(ridge2, X_test, y_test)

It looks like we are indeed improving over regular least-squares!

Instead of arbitrarily choosing alpha $ = 4$, it would be better to
use cross-validation to choose the tuning parameter alpha. We can do this using
the cross-validated ridge regression function, `RidgeCV()`. By default, the function
performs generalized cross-validation (an efficient form of LOOCV), though this can be changed using the
argument `cv`.

In [ ]:
ridgecv = make_pipeline(StandardScaler(), 
                        RidgeCV(alphas = alphas))#, 
ridgecv.fit(X_train, y_train)
ridgecv['ridgecv'].alpha_, ridgecv['ridgecv'].best_score_

Therefore, we see that the value of alpha that results in the smallest cross-validation
error is 0.57. What is the test MSE associated with this value of
alpha?

In [ ]:
ridge4 = make_pipeline(StandardScaler(), Ridge(alpha = ridgecv['ridgecv'].alpha_))
ridge4.fit(X_train, y_train)
print_result(ridge4, X_test, y_test)

This represents a further improvement over the test MSE that we got using
alpha $ = 4$. Finally, we refit our ridge regression model on the full data set,
using the value of alpha chosen by cross-validation, and examine the coefficient
estimates.

In [ ]:
ridge4.fit(X, y)
print_result(ridge4)

As expected, none of the coefficients are exactly zero - ridge regression does not
perform variable selection!

# 6.6.2 The Lasso
We saw that ridge regression with a wise choice of alpha can outperform least
squares as well as the null model on the Hitters data set. We now ask
whether the lasso can yield either a more accurate or a more interpretable
model than ridge regression. In order to fit a lasso model, we'll
use the `Lasso()` function; however, this time we'll need to include the argument `max_iter = 10000`.
Other than that change, we proceed just as we did in fitting a ridge model:

In [ ]:
lasso = make_pipeline(StandardScaler(), Lasso(max_iter = 10000))
coefs = []
test_err = []

for a in alphas:
    lasso['lasso'].set_params(alpha=a)
    lasso.fit(X_train, y_train)
    test_err.append(mean_squared_error(y_test, lasso.predict(X_test)))
    coefs.append(lasso['lasso'].coef_)
    

plt.figure(figsize=(8,8))
plt.plot(alphas, coefs, label=X.columns)
plt.legend(loc='upper right')
plt.grid()
plt.xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')

In [ ]:
plt.plot(alphas, test_err)
plt.grid()
plt.xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('test_error')

Notice that in the coefficient plot that depending on the choice of tuning
parameter, some of the coefficients are exactly equal to zero. We now
perform 10-fold cross-validation to choose the best alpha, refit the model, and compute the associated test error:

In [ ]:
lassocv = make_pipeline(StandardScaler(),
                        LassoCV(alphas = None, cv=10, max_iter = 100000))
lassocv.fit(X_train, y_train)

lasso['lasso'].set_params(alpha=lassocv['lassocv'].alpha_)
lasso.fit(X_train, y_train)
print_result(lasso, X_test, y_test)

This is substantially lower than the test set MSE of the null model and of
least squares, and only a little worse than the test MSE of ridge regression with alpha
chosen by cross-validation.

However, the lasso has a substantial advantage over ridge regression in
that the resulting coefficient estimates are sparse. Here we see that 13 of
the 19 coefficient estimates are exactly zero: